<a href="https://colab.research.google.com/github/invi-bhagyesh/study-GPT/blob/main/training_invi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/invi-bhagyesh/study-GPT.git

Cloning into 'study-GPT'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 109 (delta 28), reused 98 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 4.40 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/LLM/study-GPT

/content/drive/MyDrive/LLM/study-GPT


In [3]:
%ls

assets/  gpt2_epoch_epoch1.pt  main.py  __pycache__/  requirements.txt  train.py
data/    LICENSE               model/   README.md     training.ipynb    wandb/


In [6]:
# Optional: install wandb if you want
# !pip install wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import math
from tqdm import tqdm

# Optional
# import wandb

In [5]:
from model.gpt2 import GPT2


In [6]:
# For GPT-2
from transformers import GPT2Config
from model.gpt2 import GPT2  # assuming you have your custom GPT2 implementation

# Create config
config = GPT2Config(
    n_layer=12,    # number of transformer layers
    n_head=12,     # number of attention heads
    n_embd=768    # embedding dimension
)

# Initialize model
model = GPT2(config)

# 1. Basic parameter count
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters
total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params:,}")

Total trainable parameters: 124,439,808


In [7]:
"""
from transformers import GPT2Tokenizer
from data.dataset import TextDataset
from train import train_gpt2

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
texts = dataset["train"]["text"]  # Just get the plain text list


# Create dataset
dataset = TextDataset(texts, tokenizer)
val_dataset = TextDataset(dataset["validation"]["text"], tokenizer)

# Train the model
model = train_gpt2(
    train_dataset=dataset,
    val_dataset=val_dataset,
    n_epochs=3,
    batch_size=8,
    learning_rate=3e-4
)

# Save the model
#torch.save(model.state_dict(), 'gpt2_model.pt')
"""

'\nfrom transformers import GPT2Tokenizer\nfrom data.dataset import TextDataset\nfrom train import train_gpt2\n\n# Initialize tokenizer\ntokenizer = GPT2Tokenizer.from_pretrained(\'gpt2\')\n\nfrom datasets import load_dataset\n\ndataset = load_dataset("wikitext", "wikitext-2-raw-v1")\ntexts = dataset["train"]["text"]  # Just get the plain text list\n\n\n# Create dataset\ndataset = TextDataset(texts, tokenizer)\nval_dataset = TextDataset(dataset["validation"]["text"], tokenizer)\n\n# Train the model\nmodel = train_gpt2(\n    train_dataset=dataset,\n    val_dataset=val_dataset,\n    n_epochs=3,\n    batch_size=8,\n    learning_rate=3e-4\n)\n\n# Save the model\n#torch.save(model.state_dict(), \'gpt2_model.pt\')\n'

In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [9]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
from train import train_gpt2

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

tokenized_dataset.save_to_disk("data/tokenized_wikitext2")

train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36718 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset["train"]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 36718
})

In [11]:
total_tokens = sum([len(tokenizer(x['text'])['input_ids']) for x in dataset['train']])
print(total_tokens)

2391884


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from model.config import GPT2Config
from model.gpt2 import GPT2
from transformers import GPT2Tokenizer
from tqdm.notebook import tqdm   # NOTE: notebook-friendly tqdm
import wandb

In [13]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: invi-bhagyesh (invi-bhagyesh-manipal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [14]:
from train import train_gpt2

In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_loader = DataLoader(tokenized_dataset["train"], batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator)


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
train_gpt2(train_loader, val_loader, n_epochs=10)
# Save the model after training
torch.save(model.state_dict(), "data/gpt2_pretrained_invi.pth")
print("Model saved to data/gpt2_pretrained_invi.pth")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: invi-bhagyesh (invi-bhagyesh-manipal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/10: 100%|██████████| 4590/4590 [1:17:25<00:00,  1.01s/it]


Epoch 1: Train Loss = nan, Val Loss = 0.0336
Model saved at ./gpt2_epoch_epoch1.pt


Epoch 2/10: 100%|██████████| 4590/4590 [1:17:38<00:00,  1.01s/it]


Epoch 2: Train Loss = 0.0044, Val Loss = 0.0321
Model saved at ./gpt2_epoch_epoch2.pt


Epoch 3/10: 100%|██████████| 4590/4590 [1:17:37<00:00,  1.01s/it]


Epoch 3: Train Loss = nan, Val Loss = 0.0344
Model saved at ./gpt2_epoch_epoch3.pt


Epoch 4/10:  18%|█▊        | 832/4590 [14:03<1:03:25,  1.01s/it]

In [ ]:
# Train
model = train_gpt2(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    n_epochs=3,
    batch_size=8,
    learning_rate=3e-4
)

In [8]:
import torch

state_dict = torch.load("gpt2_epoch_epoch3.pt", map_location="cpu")
print(state_dict.keys())  # Shows the layer names and weights

odict_keys(['wte.weight', 'wpe.weight', 'blocks.0.ln_1.weight', 'blocks.0.ln_1.bias', 'blocks.0.attn.bias', 'blocks.0.attn.c_attn.weight', 'blocks.0.attn.c_attn.bias', 'blocks.0.attn.c_proj.weight', 'blocks.0.attn.c_proj.bias', 'blocks.0.ln_2.weight', 'blocks.0.ln_2.bias', 'blocks.0.mlp.0.weight', 'blocks.0.mlp.0.bias', 'blocks.0.mlp.2.weight', 'blocks.0.mlp.2.bias', 'blocks.1.ln_1.weight', 'blocks.1.ln_1.bias', 'blocks.1.attn.bias', 'blocks.1.attn.c_attn.weight', 'blocks.1.attn.c_attn.bias', 'blocks.1.attn.c_proj.weight', 'blocks.1.attn.c_proj.bias', 'blocks.1.ln_2.weight', 'blocks.1.ln_2.bias', 'blocks.1.mlp.0.weight', 'blocks.1.mlp.0.bias', 'blocks.1.mlp.2.weight', 'blocks.1.mlp.2.bias', 'blocks.2.ln_1.weight', 'blocks.2.ln_1.bias', 'blocks.2.attn.bias', 'blocks.2.attn.c_attn.weight', 'blocks.2.attn.c_attn.bias', 'blocks.2.attn.c_proj.weight', 'blocks.2.attn.c_proj.bias', 'blocks.2.ln_2.weight', 'blocks.2.ln_2.bias', 'blocks.2.mlp.0.weight', 'blocks.2.mlp.0.bias', 'blocks.2.mlp.2.we

In [14]:
import torch
from model.gpt2 import GPT2
from model.config import GPT2Config  # Ensure this matches your model definition

# Load model configuration (update this if needed)
config = GPT2Config()  # Ensure this matches how the model was originally defined
model = GPT2(config)   # Create model instance

# Load trained model weights
trained_state_dict = torch.load("gpt2_epoch_epoch3.pt", map_location="cpu")

# Load inference model weights
inference_state_dict = model.state_dict()

# Compare layer names
trained_layers = set(trained_state_dict.keys())
inference_layers = set(inference_state_dict.keys())

# Find differences
missing_in_inference = trained_layers - inference_layers
missing_in_trained = inference_layers - trained_layers

print("Layers missing in inference model:", missing_in_inference)
print("Layers missing in trained model:", missing_in_trained)

# ✅ Compare shapes of matching layers
mismatched_shapes = {
    layer: (trained_state_dict[layer].shape, inference_state_dict[layer].shape)
    for layer in (trained_layers & inference_layers)
    if trained_state_dict[layer].shape != inference_state_dict[layer].shape
}

print("Mismatched layer shapes:", mismatched_shapes)

Layers missing in inference model: set()
Layers missing in trained model: set()
Mismatched layer shapes: {}


In [15]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")  # Or your custom tokenizer
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")  # Convert to tensor

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [16]:
model.eval()  # Ensure the model is in evaluation mode

with torch.no_grad():
    for _ in range(50):  # Max 50 tokens
        logits = model(input_ids)  # Forward pass
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)  # Get most probable token
        input_ids = torch.cat([input_ids, next_token_id], dim=1)  # Append new token

        if next_token_id.item() == tokenizer.eos_token_id:  # Stop if EOS token is reached
            break

In [17]:
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

Generated Text: Once upon a time heresy heresypent WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC nm nm nm nm nm subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsidpsonspsonspsons


In [18]:
import torch
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Input text
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Ensure model is in evaluation mode
model.eval()

def sample_next_token(logits, temperature=1.0, top_k=50):
    logits = logits[:, -1, :] / temperature  # Scale by temperature
    top_k_logits, _ = torch.topk(logits, k=top_k)  # Keep top-k tokens
    probs = torch.nn.functional.softmax(top_k_logits, dim=-1)  # Convert to probabilities
    next_token = torch.multinomial(probs, num_samples=1)  # Sample from the distribution
    return next_token

# Generate text
with torch.no_grad():
    for _ in range(50):  # Limit to 50 tokens
        logits = model(input_ids)
        next_token_id = sample_next_token(logits, temperature=0.8, top_k=40)  # Adjust temperature & top-k
        input_ids = torch.cat([input_ids, next_token_id], dim=1)

        if next_token_id.item() == tokenizer.eos_token_id:
            break

# Decode output
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

Generated Text: Once upon a time7?"F,(4*=AG<C'48*75#5-$:.>0A0530A:#4-'"#@$!5&&;6DB


In [19]:
test_text = "The quick brown fox jumps over the lazy dog."
tokens = tokenizer(test_text, return_tensors="pt")

print("Tokenized IDs:", tokens["input_ids"])
print("Decoded Text:", tokenizer.decode(tokens["input_ids"][0]))

Tokenized IDs: tensor([[  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13]])
Decoded Text: The quick brown fox jumps over the lazy dog.


In [20]:
sample_text = dataset["train"][0]["text"]
print("Sample Training Text:", sample_text)

NameError: name 'dataset' is not defined

In [22]:
import torch

def generate_text(model, tokenizer, prompt, max_length=50):
    model.eval()

    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

    generated_ids = input_ids.clone()

    with torch.no_grad():
        for _ in range(max_length):
            logits = model(generated_ids)  # Forward pass
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)  # Pick highest probability token
            generated_ids = torch.cat([generated_ids, next_token], dim=1)

            if next_token.item() == tokenizer.eos_token_id:  # Stop at EOS
                break

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Example usage
prompt = "Once upon a time"
output = generate_text(model, tokenizer, prompt)
print("Generated Text:", output)

Generated Text: Once upon a time heresy heresypent WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC WTC nm nm nm nm nm subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsid subsidpsonspsonspsons


In [23]:
print(tokenizer.decode(tokenized_dataset["train"][0]["input_ids"]))

NameError: name 'tokenized_dataset' is not defined